In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
#Importing all the necessary libraries
import re
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
nltk.download("stopwords")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [54]:
import itertools
colors = itertools.cycle(["r", "b", "g"])

In [55]:
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path
import os
import glob
import math
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [56]:
import os

folder_path = '/content/drive/MyDrive/IR_ASSIGNMENT2/CSE508_Winter2023_Dataset/'

# here we are initializing a count variable.
count = 0

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    with open(file_path, 'r') as file:
        Datas = file.read()

    start_title = Datas.find("<TITLE>")+len("<TITLE>")
    end_title = Datas.find("</TITLE>")
    title = Datas[start_title:end_title].strip()

    start_text = Datas.find("<TEXT>") + len("<TEXT>")
    end_text = Datas.find("</TEXT>")
    text = Datas[start_text:end_text].strip()    
    new_Data = title + " " + text

    # here we are checking if less than five files are processed.
    if count < 5:
        print(color.BOLD+color.DARKCYAN+f"Datas of file {filename} before:\n{Datas}\n")
        print(color.BOLD+color.CYAN+f"Datas of file {filename} after:\n{new_Data}\n")
        
    # here we are incrementing the count variable.
    count = count+1

    # here we are writing the modified Datas back to the file.
    with open(file_path, 'w') as file:
        file.write(new_Data)
        
    # here we are breaking out of loop if we have processed five files.
    if count == 5:
        continue

Datas of file cranfield0351 before:
<DOC>
<DOCNO>
351
</DOCNO>
<TITLE>
thermal distributions in jeffrey-hamel flows between nonparallel plane
walls .
</TITLE>
<AUTHOR>
millsaps, k. and pohlhausen, k.
</AUTHOR>
<BIBLIO>
j. aero. sc. v. 20. march 1953, pp 187-196 .
</BIBLIO>
<TEXT>
the authors give the exact solution for the thermal distributions for
the steady laminar flow of a viscous incompressible fluid between nonparallel
plane walls held at a constant temperature . the velocity
profiles are determined with the aid of jacobian elliptic functions by
using the jeffery-hamel solution of the hydrodynamic problem . it is
shown that in this special case the energy equation giving the temperature
profiles can be reduced to an ordinary linear differential
equation with variable coefficients . after the introduction of dimensionless
parameters, numerical solutions are given for diverging and
converging channels with total openings of 10degree for the possible
combinations of three reynolds n

In [57]:
# Here we are bring out all the files.
f_all=os.listdir('/content/drive/MyDrive/IR_ASSIGNMENT2/CSE508_Winter2023_Dataset/')

In [58]:
#here we are printing all the filenames
print(f_all)

['cranfield0351', 'cranfield0525', 'cranfield0792', 'cranfield0332', 'cranfield0024', 'cranfield0109', 'cranfield0158', 'cranfield0288', 'cranfield0556', 'cranfield1190', 'cranfield0373', 'cranfield0135', 'cranfield0960', 'cranfield0499', 'cranfield1227', 'cranfield0999', 'cranfield0078', 'cranfield0034', 'cranfield0916', 'cranfield0422', 'cranfield0611', 'cranfield1286', 'cranfield0633', 'cranfield0851', 'cranfield0401', 'cranfield0542', 'cranfield1001', 'cranfield0679', 'cranfield0653', 'cranfield0087', 'cranfield0727', 'cranfield1125', 'cranfield1233', 'cranfield0974', 'cranfield0290', 'cranfield0730', 'cranfield0280', 'cranfield0635', 'cranfield0225', 'cranfield0605', 'cranfield0857', 'cranfield1099', 'cranfield0215', 'cranfield0032', 'cranfield0534', 'cranfield0150', 'cranfield0008', 'cranfield0772', 'cranfield0337', 'cranfield0967', 'cranfield0646', 'cranfield0331', 'cranfield0430', 'cranfield0196', 'cranfield1261', 'cranfield0016', 'cranfield1358', 'cranfield0854', 'cranfield017

In [59]:
# here we are creating the dictionary which has all the name of files with their respective Datas

dictionaryFC={'filename':[],'Data':[]}
for i in f_all:
  document1 = open("/content/drive/MyDrive/IR_ASSIGNMENT2/CSE508_Winter2023_Dataset/"+i,"r",encoding='utf-8',errors='ignore')
  dictionaryFC['filename'].append(i)
  text=document1.read()
  # print(text)
  dictionaryFC['Data'].append(text)

In [60]:
# here we are creating a df which has filename and Data as the column names
FileData=pd.DataFrame(dictionaryFC)

In [61]:
# printing the dataframe
print(FileData)

           filename                                               Data
0     cranfield0351  thermal distributions in jeffrey-hamel flows b...
1     cranfield0525  on hypersonic viscous flow over an insulated f...
2     cranfield0792  some low speed problems of high speed aircraft...
3     cranfield0332  similitude of hypersonic real-gas flows over s...
4     cranfield0024  theory of stagnation point heat transfer in di...
...             ...                                                ...
1395  cranfield0701  numerical determination of indical lift of a t...
1396  cranfield1103  pressures, densities and temperatures in the u...
1397  cranfield0882  the variation of gust frequency with gust velo...
1398  cranfield0195  correlation of theoretical and photo-thermoela...
1399  cranfield1212  effect of uniformly distributed roughness on t...

[1400 rows x 2 columns]


In [62]:
# here our preprocessing starts

In [63]:
# here we are performing tokenization on the text 
wordToken = nltk.tokenize.WhitespaceTokenizer()

In [64]:

def tokenized_text(text):
    return wordToken.tokenize(text)

In [65]:
# here we are lowercasing the text 
FileData['Data']=FileData['Data'].str.lower()
print(color.BOLD+color.DARKCYAN+"After lowercase : ")
print(FileData)


After lowercase : 
           filename                                               Data
0     cranfield0351  thermal distributions in jeffrey-hamel flows b...
1     cranfield0525  on hypersonic viscous flow over an insulated f...
2     cranfield0792  some low speed problems of high speed aircraft...
3     cranfield0332  similitude of hypersonic real-gas flows over s...
4     cranfield0024  theory of stagnation point heat transfer in di...
...             ...                                                ...
1395  cranfield0701  numerical determination of indical lift of a t...
1396  cranfield1103  pressures, densities and temperatures in the u...
1397  cranfield0882  the variation of gust frequency with gust velo...
1398  cranfield0195  correlation of theoretical and photo-thermoela...
1399  cranfield1212  effect of uniformly distributed roughness on t...

[1400 rows x 2 columns]


In [66]:
#here we are removing special characters and punctuation mark.
FileData['Data'] = FileData['Data'].str.replace('[^a-zA-Z0-9]',' ')
FileData['Data'] = FileData['Data'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
print(color.BOLD+color.DARKCYAN+"After removing special characters and punctuation marks : ")
print(FileData)


After removing special characters and punctuation marks : 
           filename                                               Data
0     cranfield0351  thermal distributions in jeffrey hamel flows b...
1     cranfield0525  on hypersonic viscous flow over an insulated f...
2     cranfield0792  some low speed problems of high speed aircraft...
3     cranfield0332  similitude of hypersonic real gas flows over s...
4     cranfield0024  theory of stagnation point heat transfer in di...
...             ...                                                ...
1395  cranfield0701  numerical determination of indical lift of a t...
1396  cranfield1103  pressures  densities and temperatures in the u...
1397  cranfield0882  the variation of gust frequency with gust velo...
1398  cranfield0195  correlation of theoretical and photo thermoela...
1399  cranfield1212  effect of uniformly distributed roughness on t...

[1400 rows x 2 columns]


In [67]:
#here we will remove all the stopwords.
stop_word = stopwords.words('english')
FileData['Data'] = FileData['Data'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_word)]))
print(color.BOLD+color.DARKCYAN+"After removing stopwords : ")
print(FileData)


After removing stopwords : 
           filename                                               Data
0     cranfield0351  thermal distributions jeffrey hamel flows nonp...
1     cranfield0525  hypersonic viscous flow insulated flat plate s...
2     cranfield0792  low speed problems high speed aircraft first p...
3     cranfield0332  similitude hypersonic real gas flows slender b...
4     cranfield0024  theory stagnation point heat transfer dissocia...
...             ...                                                ...
1395  cranfield0701  numerical determination indical lift two dimen...
1396  cranfield1103  pressures densities temperatures upper atmosph...
1397  cranfield0882  variation gust frequency gust velocity altitud...
1398  cranfield0195  correlation theoretical photo thermoelastic re...
1399  cranfield1212  effect uniformly distributed roughness turbule...

[1400 rows x 2 columns]


In [68]:
#here we are performing tokenization.
FileData['Data_tokens']=FileData.Data.apply(tokenized_text)
print(color.BOLD+color.DARKCYAN+"After tokenization : ")
print(FileData)

After tokenization : 
           filename                                               Data  \
0     cranfield0351  thermal distributions jeffrey hamel flows nonp...   
1     cranfield0525  hypersonic viscous flow insulated flat plate s...   
2     cranfield0792  low speed problems high speed aircraft first p...   
3     cranfield0332  similitude hypersonic real gas flows slender b...   
4     cranfield0024  theory stagnation point heat transfer dissocia...   
...             ...                                                ...   
1395  cranfield0701  numerical determination indical lift two dimen...   
1396  cranfield1103  pressures densities temperatures upper atmosph...   
1397  cranfield0882  variation gust frequency gust velocity altitud...   
1398  cranfield0195  correlation theoretical photo thermoelastic re...   
1399  cranfield1212  effect uniformly distributed roughness turbule...   

                                            Data_tokens  
0     [thermal, distributions, 

In [69]:
FileData

,filename,Data,Data_tokens
0,cranfield0351,thermal distributions jeffrey hamel flows nonp...,"[thermal, distributions, jeffrey, hamel, flows..."
1,cranfield0525,hypersonic viscous flow insulated flat plate s...,"[hypersonic, viscous, flow, insulated, flat, p..."
2,cranfield0792,low speed problems high speed aircraft first p...,"[low, speed, problems, high, speed, aircraft, ..."
3,cranfield0332,similitude hypersonic real gas flows slender b...,"[similitude, hypersonic, real, gas, flows, sle..."
4,cranfield0024,theory stagnation point heat transfer dissocia...,"[theory, stagnation, point, heat, transfer, di..."
...,...,...,...
1395,cranfield0701,numerical determination indical lift two dimen...,"[numerical, determination, indical, lift, two,..."
1396,cranfield1103,pressures densities temperatures upper atmosph...,"[pressures, densities, temperatures, upper, at..."
1397,cranfield0882,variation gust frequency gust velocity altitud...,"[variation, gust, frequency, gust, velocity, a..."
1398,cranfield0195,correlation theoretical photo thermoelastic re...,"[correlation, theoretical, photo, thermoelasti..."


In [70]:
# here we are creating dictionary which stores the frequency of tokens and we are also creating a list of unique tokens.
TokenCount={}
distinctiveTokens=[]

for tokens in FileData['Data_tokens']:
  for word in tokens:
          if(word in TokenCount.keys()):
              TokenCount[word] = TokenCount[word] + 1
          else:
              TokenCount[word] = 1
              distinctiveTokens.append(word)

In [71]:
FileData

,filename,Data,Data_tokens
0,cranfield0351,thermal distributions jeffrey hamel flows nonp...,"[thermal, distributions, jeffrey, hamel, flows..."
1,cranfield0525,hypersonic viscous flow insulated flat plate s...,"[hypersonic, viscous, flow, insulated, flat, p..."
2,cranfield0792,low speed problems high speed aircraft first p...,"[low, speed, problems, high, speed, aircraft, ..."
3,cranfield0332,similitude hypersonic real gas flows slender b...,"[similitude, hypersonic, real, gas, flows, sle..."
4,cranfield0024,theory stagnation point heat transfer dissocia...,"[theory, stagnation, point, heat, transfer, di..."
...,...,...,...
1395,cranfield0701,numerical determination indical lift two dimen...,"[numerical, determination, indical, lift, two,..."
1396,cranfield1103,pressures densities temperatures upper atmosph...,"[pressures, densities, temperatures, upper, at..."
1397,cranfield0882,variation gust frequency gust velocity altitud...,"[variation, gust, frequency, gust, velocity, a..."
1398,cranfield0195,correlation theoretical photo thermoelastic re...,"[correlation, theoretical, photo, thermoelasti..."


In [72]:
# here we are taking the users input through the function , pre-processing it and then returning the preprocessed query.  
def EnterQuery():
  query=input('Enter your query :')
  query=query.lower()
  query = re.sub('[^a-zA-Z0-9]', ' ', query)
  stop_word = stopwords.words('english')
  query=query.split()
  query_token = [word for word in query if not word in stop_word]
  return query_token

In [73]:

try:
  comparisons=0
  query=EnterQuery()
  query=set(query)
  top_five={}
  for i in range(FileData.shape[0]):
    tokens=set(FileData.iloc[i,2])
    intersec=query.intersection(tokens)
    union=query.union(tokens)
    jaccard=len(intersec)/len(union)
    top_five[jaccard]=FileData.iloc[i,0]
  answer=sorted(top_five.items(),reverse=True)
  print(color.BOLD+color.DARKCYAN+"The list of top-10 document names retrieved:\n ")
  count=0
  for k,v in answer:
    if(count>=10):
      break
    print(color.BOLD+color.BLUE+f'{v}: Jaccard Coefficient: {k}')
    count=count+1
except:
  print(color.BOLD+color.RED+'Enter Valid Input!!!')

Enter your query :numerical determination indical lift
The list of top-10 document names retrieved:
 
cranfield0702: Jaccard Coefficient: 0.07407407407407407
cranfield0322: Jaccard Coefficient: 0.06666666666666667
cranfield0320: Jaccard Coefficient: 0.058823529411764705
cranfield0728: Jaccard Coefficient: 0.05263157894736842
cranfield1219: Jaccard Coefficient: 0.046875
cranfield0701: Jaccard Coefficient: 0.041666666666666664
cranfield0250: Jaccard Coefficient: 0.04
cranfield1256: Jaccard Coefficient: 0.03571428571428571
cranfield0379: Jaccard Coefficient: 0.03333333333333333
cranfield0235: Jaccard Coefficient: 0.03278688524590164


In [74]:

# here we are creating a meta document dictionary, which is nested.
meta_doc_dict={}
# here we are creating a list of unique tokens.
distinctiveTokens=[]
for i in range(FileData.shape[0]):
  token_list=FileData.iloc[i,2]
  temp_dict={}
  for j in token_list:
    if(j in temp_dict):
      temp_dict[j]+=1
    else:
      temp_dict[j]=1
      distinctiveTokens.append(j)
  meta_doc_dict[FileData.iloc[i,0]]=temp_dict


In [75]:
# here we are printing meta document dictionary.
meta_doc_dict

{'cranfield0351': {'thermal': 2,
  'distributions': 2,
  'jeffrey': 1,
  'hamel': 2,
  'flows': 1,
  'nonparallel': 2,
  'plane': 2,
  'walls': 2,
  'authors': 1,
  'give': 1,
  'exact': 1,
  'solution': 2,
  'steady': 1,
  'laminar': 1,
  'flow': 1,
  'viscous': 1,
  'incompressible': 1,
  'fluid': 1,
  'held': 1,
  'constant': 1,
  'temperature': 2,
  'velocity': 1,
  'profiles': 2,
  'determined': 1,
  'aid': 1,
  'jacobian': 1,
  'elliptic': 1,
  'functions': 1,
  'using': 1,
  'jeffery': 1,
  'hydrodynamic': 1,
  'problem': 1,
  'shown': 1,
  'special': 1,
  'case': 1,
  'energy': 1,
  'equation': 2,
  'giving': 1,
  'reduced': 1,
  'ordinary': 1,
  'linear': 1,
  'differential': 1,
  'variable': 1,
  'coefficients': 1,
  'introduction': 1,
  'dimensionless': 1,
  'parameters': 1,
  'numerical': 1,
  'solutions': 1,
  'given': 1,
  'diverging': 1,
  'converging': 1,
  'channels': 1,
  'total': 1,
  'openings': 1,
  '10degree': 1,
  'possible': 1,
  'combinations': 1,
  'three': 1,

In [76]:
# here we are creating a posting list, which is a dictionary.
posting_list={}

for word in distinctiveTokens:
    posting_list[word] = []

for i in range(len(FileData)):
    tokens = FileData.iloc[i,2]
    for word in tokens:
        if(i not in posting_list[word]):
            posting_list[word].append(i) 

In [77]:
# here we are printing the posting list.
posting_list

{'thermal': [0,
  3,
  11,
  41,
  49,
  50,
  70,
  98,
  113,
  138,
  172,
  174,
  193,
  215,
  223,
  244,
  258,
  289,
  305,
  306,
  316,
  328,
  329,
  330,
  349,
  362,
  375,
  423,
  430,
  453,
  465,
  499,
  514,
  515,
  548,
  553,
  556,
  563,
  573,
  582,
  591,
  593,
  602,
  624,
  627,
  631,
  638,
  639,
  644,
  647,
  650,
  652,
  698,
  708,
  710,
  731,
  767,
  771,
  780,
  803,
  828,
  830,
  906,
  912,
  920,
  944,
  945,
  963,
  986,
  991,
  1006,
  1072,
  1076,
  1114,
  1118,
  1123,
  1146,
  1159,
  1221,
  1223,
  1275,
  1284,
  1313,
  1347,
  1369,
  1398],
 'distributions': [0,
  27,
  37,
  38,
  52,
  53,
  54,
  61,
  63,
  65,
  66,
  68,
  70,
  76,
  88,
  97,
  108,
  149,
  151,
  174,
  175,
  183,
  199,
  207,
  210,
  217,
  226,
  234,
  244,
  255,
  258,
  277,
  288,
  289,
  290,
  292,
  298,
  305,
  329,
  353,
  354,
  363,
  367,
  375,
  385,
  386,
  400,
  402,
  416,
  426,
  429,
  436,
  467,
  470,
  

In [78]:
#here we are genrating the idf values corresponding to unique tokens.
idf_values={}
for i in distinctiveTokens:
  # print(i)
  # print("length:",len(posting_list[i]))
  # print(FileData.shape[0]/(len(posting_list[i])+1))
  idf=math.log(FileData.shape[0]/(len(posting_list[i])+1))
  idf_values[i]=idf

In [79]:
# FileData.shape[0]

In [80]:
# here we printing the idf values
idf_values

{'thermal': 2.7783193969487665,
 'distributions': 2.281382885343443,
 'jeffrey': 6.551080335043404,
 'hamel': 6.551080335043404,
 'flows': 2.316973830446145,
 'nonparallel': 6.145615226935241,
 'plane': 2.7668907011251433,
 'walls': 3.7178669909871886,
 'authors': 3.5553480614894135,
 'give': 3.0247198104272432,
 'exact': 2.571398681141444,
 'solution': 1.6238266498862,
 'steady': 2.6094985273737143,
 'laminar': 1.784642001459191,
 'flow': 0.690294111577539,
 'viscous': 2.4320431602309327,
 'incompressible': 2.3538783873815965,
 'fluid': 2.207274913189721,
 'held': 4.605170185988092,
 'constant': 2.1382820417027695,
 'temperature': 1.7227665977411035,
 'velocity': 1.595253277442144,
 'profiles': 3.039534896212384,
 'determined': 2.302585092994046,
 'aid': 4.153185062245034,
 'jacobian': 6.551080335043404,
 'elliptic': 3.810240311118204,
 'functions': 2.8747796631363283,
 'using': 2.2137895942109145,
 'jeffery': 6.551080335043404,
 'hydrodynamic': 4.75932086581535,
 'problem': 1.7719568

In [81]:
# Binary Weighting scheme
print(color.BOLD+color.BLUE+'Binary Weighting Scheme')

Binary Weighting Scheme


In [82]:
matrix_tf_idf_binary=np.zeros((FileData.shape[0],len(distinctiveTokens)))
value=1
for i in range(FileData.shape[0]):
  doc=meta_doc_dict[FileData.iloc[i,0]]
  tokens=list(doc.keys())
  for word in tokens:
    tf=value
    idf=idf_values[word]
    overall_score=tf*idf
    matrix_tf_idf_binary[i,distinctiveTokens.index(word)]=overall_score

In [83]:
print(matrix_tf_idf_binary)

[[2.7783194  2.28138289 6.55108034 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [2.7783194  2.28138289 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [84]:
# Raw Count Weighting scheme
print(color.BOLD+color.BLUE+'Raw Count Weighting Scheme')

Raw Count Weighting Scheme


In [85]:
matrix_tf_idf_raw_count=np.zeros((FileData.shape[0],len(distinctiveTokens)))
value=1
for i in range(FileData.shape[0]):
  doc=meta_doc_dict[FileData.iloc[i,0]]
  tokens=list(doc.keys())
  for word in tokens:
    tf=doc[word]
    idf=idf_values[word]
    overall_score=tf*idf
    matrix_tf_idf_raw_count[i,distinctiveTokens.index(word)]=overall_score

In [86]:
print(matrix_tf_idf_raw_count)

[[ 5.55663879  4.56276577  6.55108034 ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [13.89159698  2.28138289  0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [87]:
# Term frequency Weighting scheme
print(color.BOLD+color.BLUE+'Term frequency Weighting Scheme')

Term frequency Weighting Scheme


In [88]:
matrix_tf_idf_term_freq=np.zeros((FileData.shape[0],len(distinctiveTokens)))
value=1
for i in range(FileData.shape[0]):
  doc=meta_doc_dict[FileData.iloc[i,0]]
  tokens=list(doc.keys())
  overall_tf=0
  for w in tokens:
    overall_tf+=doc[w]
  for word in tokens:
    tf=doc[word]/overall_tf
    idf=idf_values[word]
    overall_score=tf*idf
    matrix_tf_idf_term_freq[i,distinctiveTokens.index(word)]=overall_score

In [89]:
print(matrix_tf_idf_term_freq)

[[0.07508971 0.061659   0.08852811 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.13891597 0.02281383 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [90]:
# log normalization Weighting scheme
print(color.BOLD+color.BLUE+'log normalization Weighting Scheme')

log normalization Weighting Scheme


In [91]:
matrix_tf_idf_log_norm=np.zeros((FileData.shape[0],len(distinctiveTokens)))
value=1
for i in range(FileData.shape[0]):
  doc=meta_doc_dict[FileData.iloc[i,0]]
  tokens=list(doc.keys())
  for word in tokens:
    tf=math.log(doc[word]+1)
    idf=idf_values[word]
    overall_score=tf*idf
    matrix_tf_idf_log_norm[i,distinctiveTokens.index(word)]=overall_score

In [92]:
print(matrix_tf_idf_log_norm)

[[3.05229583 2.50635527 4.54086286 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [4.97808009 1.58133411 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [93]:
# Double Normalization Weighting scheme
print(color.BOLD+color.BLUE+'Double Normalization Weighting Scheme')

Double Normalization Weighting Scheme


In [94]:
matrix_tf_idf_double_norm=np.zeros((FileData.shape[0],len(distinctiveTokens)))
value=1
for i in range(FileData.shape[0]):
  doc=meta_doc_dict[FileData.iloc[i,0]]
  tokens=list(doc.keys())
  tf_max=0
  for w in tokens:
    temp=doc[w]
    if(tf_max<temp):
      tf_max=temp
  for word in tokens:
    tf=0.5*(doc[word]/tf_max)+0.5
    idf=idf_values[word]
    overall_score=tf*idf
    matrix_tf_idf_double_norm[i,distinctiveTokens.index(word)]=overall_score

In [95]:
print(matrix_tf_idf_double_norm)

[[2.7783194  2.28138289 4.91331025 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [2.38141663 1.30364736 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [96]:
len_df=FileData.shape[0]
query=EnterQuery()
final_ans={}
for x in range(len_df):
  score=0
  doc=matrix_tf_idf_binary[x,:]
  for token in query:
    try:
      score+=doc[distinctiveTokens.index(token)]
    except:
      pass
  final_ans[FileData.iloc[x,0]]=score

final_sorted_ans=dict(sorted(final_ans.items(),key=lambda item:item[1],reverse=True))
count=0
print(color.BOLD+color.DARKCYAN+"Using Binary Weighting Scheme")
for k,v in final_sorted_ans.items():
  count+=1

  print(f'{count}: {k}    score: {v}')
  if(count>=5):
    break



Enter your query :numerical determination indical lift
Using Binary Weighting Scheme
1: cranfield0702    score: 13.483234355806001
2: cranfield0701    score: 13.483234355806001
3: cranfield0699    score: 8.19688589164838
4: cranfield1219    score: 7.625301201322541
5: cranfield0379    score: 5.522737242221851


In [97]:
len_df=FileData.shape[0]
query=EnterQuery()
final_ans={}
for x in range(len_df):
  score=0
  doc=matrix_tf_idf_raw_count[x,:]
  for token in query:
    try:
      score+=doc[distinctiveTokens.index(token)]
    except:
      pass
  final_ans[FileData.iloc[x,0]]=score

final_sorted_ans=dict(sorted(final_ans.items(),key=lambda item:item[1],reverse=True))
count=0
print(color.BOLD+color.DARKCYAN+"Using Raw Count Weighting Scheme")
for k,v in final_sorted_ans.items():
  count+=1

  print(f'{count}: {k}    score: {v}')
  if(count>=5):
    break



Enter your query :numerical determination indical lift
Using Raw Count Weighting Scheme
1: cranfield0701    score: 39.21171446462013
2: cranfield0699    score: 26.908507788967743
3: cranfield1380    score: 23.389527371649205
4: cranfield0702    score: 20.50009256730076
5: cranfield1291    score: 18.711621897319365


In [98]:
len_df=FileData.shape[0]
query=EnterQuery()
final_ans={}
for x in range(len_df):
  score=0
  doc=matrix_tf_idf_term_freq[x,:]
  for token in query:
    try:
      score+=doc[distinctiveTokens.index(token)]
    except:
      pass
  final_ans[FileData.iloc[x,0]]=score

final_sorted_ans=dict(sorted(final_ans.items(),key=lambda item:item[1],reverse=True))
count=0
print(color.BOLD+color.DARKCYAN+"Using Term Frequency Weighting Scheme")
for k,v in final_sorted_ans.items():
  count+=1

  print(f'{count}: {k}    score: {v}')
  if(count>=5):
    break



Enter your query :numerical determination indical lift
Using Term Frequency Weighting Scheme
1: cranfield0701    score: 0.24355102151937966
2: cranfield0699    score: 0.22612191419300623
3: cranfield0702    score: 0.2029712135376313
4: cranfield0698    score: 0.19726107421872824
5: cranfield1256    score: 0.17991944132037851


In [99]:
len_df=FileData.shape[0]
query=EnterQuery()
final_ans={}
for x in range(len_df):
  score=0
  doc=matrix_tf_idf_log_norm[x,:]
  for token in query:
    try:
      score+=doc[distinctiveTokens.index(token)]
    except:
      pass
  final_ans[FileData.iloc[x,0]]=score

final_sorted_ans=dict(sorted(final_ans.items(),key=lambda item:item[1],reverse=True))
count=0
print(color.BOLD+color.DARKCYAN+"Using Log Normalization Weighting Scheme")
for k,v in final_sorted_ans.items():
  count+=1

  print(f'{count}: {k}    score: {v}')
  if(count>=5):
    break



Enter your query :numerical determination indical lift
Using Log Normalization Weighting Scheme
1: cranfield0701    score: 13.723922703651153
2: cranfield0702    score: 11.489026593911822
3: cranfield0699    score: 9.446047555752402
4: cranfield1219    score: 6.906694523845914
5: cranfield1380    score: 5.608563711746573


In [100]:
len_df=FileData.shape[0]
query=EnterQuery()
final_ans={}
for x in range(len_df):
  score=0
  doc=matrix_tf_idf_double_norm[x,:]
  for token in query:
    try:
      score+=doc[distinctiveTokens.index(token)]
    except:
      pass
  final_ans[FileData.iloc[x,0]]=score

final_sorted_ans=dict(sorted(final_ans.items(),key=lambda item:item[1],reverse=True))
count=0
print(color.BOLD+color.DARKCYAN+"Using Double Normalization Weighting Scheme")
for k,v in final_sorted_ans.items():
  count+=1

  print(f'{count}: {k}    score: {v}')
  if(count>=5):
    break



Enter your query :numerical determination indical lift
Using Double Normalization Weighting Scheme
1: cranfield0701    score: 8.375438613928837
2: cranfield0702    score: 7.766621806268039
3: cranfield0699    score: 5.593360045211287
4: cranfield1219    score: 5.3505514351178185
5: cranfield0322    score: 3.9647613481182153
